In [132]:
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons

In [133]:
from sklearn.datasets import make_classification

In [134]:
X, y = make_classification(n_samples=500, random_state=1)
y = np.where(y==0,-1,1)  #change our y to be -1 if it is 0, otherwise 1

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

In [135]:
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

In [136]:
np.log((1 - 1e-308) / 1e-308 ) #the smallest value of the error function that numpy can calculate

709.1962086421661

In [137]:
m = X_train.shape[0]
S = 20
stump_params = {'max_depth': 1, 'max_leaf_nodes': 2}
models = [DecisionTreeClassifier(**stump_params) for _ in range(S)]

#initially, we set our weight to 1/m
W = np.full(m, 1/m)

#keep collection of a_j
a_js = np.zeros(S)

#threshold for the calculation of the error
epsilon = 1e-50

#definition of eta
eta = 1/2

for j, model in enumerate(models):
    
    #train weak learner
    model.fit(X_train, y_train, sample_weight = W)
    
    #compute the errors
    yhat = model.predict(X_train) 
    err = W[(yhat != y_train)].sum()
        
    #compute the predictor weight a_j
    #if predictor is doing well, a_j will be big
    if err < epsilon :
        err += 1e-20
    a_j = np.log ((1 - err) / err) * eta
    a_js[j] = a_j
    
    #update sample weight; divide sum of W to normalize
    W = (W * np.exp(-a_j * y_train * yhat)) 
    W = W / sum (W)
    
        
#make weighted predictions
Hx = 0
for i, model in enumerate(models):
    yhat = model.predict(X_test)
    Hx += a_js[i] * yhat
    
yhat = np.sign(Hx)

print(classification_report(y_test, yhat))

              precision    recall  f1-score   support

          -1       0.96      0.97      0.97        79
           1       0.97      0.96      0.96        71

    accuracy                           0.97       150
   macro avg       0.97      0.97      0.97       150
weighted avg       0.97      0.97      0.97       150



AdaBoost model seems solid to the changes of values for eta when this one is inferior of 1, but sensitive when the value of eta is superior than 1 

In [138]:
class Stump:

    def __init__(self, eta):
        self.threshold = None
        self.feature = None
    
    def fit(self, X, y, sample_weight) :
        n_samples, n_features = X.shape
        best_err = (sample_weight[(-y != y)].sum())*(1 / n_samples) #initialization with the biggest error we could find
        for feature in range(n_features) :
            sort_feature_sample = np.sort(X[:, feature])
            for sample in range(n_samples - 1) :
                if sort_feature_sample[sample] == sort_feature_sample[sample + 1] :
                    continue
                else :
                    threshold = (sort_feature_sample[sample] + sort_feature_sample[sample + 1]) / 2
                    yhat = np.ones(n_samples)
                    yhat[ X[:, feature] < threshold ] = -1
                    err = (sample_weight[(yhat != y)].sum())*(1 / n_samples)
                
                    if err < best_err :
                        best_err = err
                        threshold_ix = threshold
                        feature_ix = feature
        
        self.threshold = threshold_ix
        self.feature = feature_ix
        
    def predict(self, X) :
        if self.feature is not None :
            yhat = np.ones(X.shape[0])
            yhat[ X[:, self.feature] < self.threshold ] = -1
            return yhat
        else :
            raise ValueError('There is something wrong')
        

In [139]:
class AdaBoost :
    def __init__(self, S, eta):
        self.S = S
        self.eta = eta
    
    def fit(self, X, y):
        m, n = X.shape
        
        W = np.full(m, 1/m)
        classifiers = []
        a_js = np.zeros(self.S)
        
        for k in range(self.S) :
            stump = Stump(self.eta)
            stump.fit(X, y, W)
            yhat = stump.predict(X)
            err = W[(yhat != y)].sum()
            
            if err < epsilon :
                err += 1e-20
            a_j = np.log ((1 - err) / err) * self.eta
            a_js[k] = a_j

            #update sample weight; divide sum of W to normalize
            W = (W * np.exp(-a_j * y * yhat)) 
            W = W / sum (W)
            classifiers.append(stump)
        self.a_js = a_js
        self.classifiers = classifiers
    
    def predict(self, X):
        Hx = 0
        a_js = self.a_js
        for i, model in enumerate(self.classifiers):
            yhat = model.predict(X_test)
            Hx += a_js[i] * yhat

        yhat = np.sign(Hx)
        return yhat

In [140]:
exp = AdaBoost(20, 1/2)
exp.fit(X_train, y_train)
yhat = exp.predict(X_test)

print(classification_report(y_test, yhat))

              precision    recall  f1-score   support

          -1       0.96      0.99      0.97        79
           1       0.99      0.96      0.97        71

    accuracy                           0.97       150
   macro avg       0.97      0.97      0.97       150
weighted avg       0.97      0.97      0.97       150

